In [3]:
flatten_leaves = lambda d: {k: v for sd in d.values() for k, v in (flatten_leaves(sd).items() if isinstance(sd, dict) else sd.items())}


In [5]:
data = {
    "JSONLoader": {
        "required": {
            "jq_schema": None,
        },
        "optional": {
            "content_key": None,
            "is_content_key_jq_parsable": False,
            "metadata_func": None,
            "text_content": True,
            "json_lines": False,
        }
    }
}
kwargs = {**data['JSONLoader']['required'], **data['JSONLoader']['optional']}

In [6]:
kwargs

{'jq_schema': None,
 'content_key': None,
 'is_content_key_jq_parsable': False,
 'metadata_func': None,
 'text_content': True,
 'json_lines': False}

In [1]:
d = {
    1: 12,
    2: 23,
    3: "foo"
}

In [2]:
s = d[3]

del d[3]
print(f"s: {s}")
print(f"d: {d}")

s: foo
d: {1: 12, 2: 23}


In [1]:
import chromadb

In [3]:
chroma_client = chromadb.Client()

In [4]:
collection = chroma_client.create_collection(name="my_collection")

In [5]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


/Users/shauryapednekar/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:17<00:00, 4.63MiB/s]


In [6]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'distances': [[1.0403728485107422, 1.2430635690689087]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None}


In [4]:
from langchain_community.vectorstores import FAISS, Chroma
from langchain_huggingface import HuggingFaceEmbeddings


In [13]:
embeddings = HuggingFaceEmbeddings(**{
        "model_name": "sentence-transformers/all-mpnet-base-v2",
        "model_kwargs": {"device": "cuda"},
        "encode_kwargs": {"normalize_embeddings": False},
    })
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

/home/shaurya/foo/src/ingestion-pipeline/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
docs = new_db.similarity_search("Relationship between trump and farmers?")

In [15]:
docs

[Document(page_content="LIESMAN: UNDERSTOOD --\nMNUCHIN: AND WE ALSO ANNOUNCED YESTERDAY AS IT RELATES TO AGRICULTURE, WE'RE GOING TO PROTECT OUR FARMERS. THE PRESIDENT INSTRUCTED SECRETARY PERDUE TO WORK WITHTHE REST OF THE CABINET TO PROTECT OUR FARMERS FROM UNFAIR TARGETING.\nLIESMAN: ONE OTHER QUESTION OF ABIDING CONCERN TO THE MARKET HOW POLICY IS FORMED IN THE ADMINISTRATION, CAN YOU TELL US -- DID YOU KNOW ABOUT THE PRESIDENTS THREAT OF $100 BILLION OF TARIFFS BEFORE HE ANNOUNCED IT?", metadata={'language': 'english', 'url': 'http://www.cnbc.com/2018/04/06/cnbc-exclusive-cnbc-transcript-u-s-treasury-secretary-steven-mnuchin-on-cnbcs-power-lunch-today.html', 'published': '2018-04-06T23:25:00.000+03:00', 'title': 'CNBC Exclusive: CNBC Transcript: U.S. Treasury Secretary Steven Mnuchin on CNBC’s “Power Lunch” Today', 'source': '../unzipped/financial_dataset/2018_04_112b52537b67659ad3609a234388c50a/news_0013370.json'}),
 Document(page_content="DAILY INVOLVED IN THESE CONVERSATIONS W

In [1]:
from enhanced_document import EnhancedDocument

In [2]:
# Example instantiation
page_content = "This is an example document content."
metadata = {
    "author": "John Doe",
    "title": "Example Document",
    "date": "2024-06-19"
}

enhanced_document = EnhancedDocument(page_content=page_content, metadata=metadata)


In [3]:
print(enhanced_document.content_hash)

None


In [1]:
from __future__ import annotations

import hashlib
import json
import logging
import uuid
from typing import Any, Dict, Optional

from langchain_core.documents import Document
from pydantic.v1 import root_validator

# Magic UUID to use as a namespace for hashing.
# Used to try and generate a unique UUID for each document
# from hashing the document content and metadata.
NAMESPACE_UUID = uuid.UUID(int=1984)


def _hash_string_to_uuid(input_string: str) -> uuid.UUID:
    """Hashes a string and returns the corresponding UUID."""
    hash_value = hashlib.sha1(input_string.encode("utf-8")).hexdigest()
    return uuid.uuid5(NAMESPACE_UUID, hash_value)

In [2]:
_hash_string_to_uuid('May 7, 2018 / 6:58 PM / Updated 19 minutes ago Trump to announce decision on Iran nuclear deal on Tuesday Reuters Staff 2 Min Read')

UUID('c54c0512-bd37-5b5a-8c8d-969458fbc79b')

In [3]:
_hash_string_to_uuid('May 7, 2018 / 6:58 PM / Updated 19 minutes ago Trump to announce decision on Iran nuclear deal on Tuesday Reuters Staff 2 Min Read\nWASHINGTON (Reuters) - U.S. President Donald Trump said he would announce a decision on the future of the Iran nuclear deal on Tuesday as European countries piled pressure on Washington to remain in the 2015 agreement. President Donald Trump speaks during a roundtable discussion on tax reform at the Cleveland Public Auditorium in Cleveland, Ohio, U.S., May 5, 2018. REUTERS/Aaron P. Bernstein\nTrump has threatened to withdraw from the deal, which provided Iran with relief from economic sanctions in exchange for limiting its uranium enrichment capacity, unless European signatories to the accord fix what he has called its shortcomings.\n“I will be announcing my decision on the Iran Deal tomorrow from the White House at 2:00pm,” Trump said in a Twitter post on Monday.\nTrump has until May 12 to decide whether to reintroduce U.S. sanctions on Iran, which would deal a heavy blow to the agreement.\nUnder the agreement with the United States, France, Germany, Britain, Russia and China, Iran strictly limited uranium enrichment capacity to try to show that it was not trying to develop atomic bombs. In exchange, Iran received relief from economic sanctions.\nIranian President Hassan Rouhani hinted on Monday that Iran could remain in the nuclear accord even if the United States dropped out, but said that Tehran would fiercely resist U.S. pressure to limit its influence in the Middle East.\nBritain, France and Germany remain committed to the accord and, in an effort to address U.S. complaints, want to open talks on Iran’s ballistic missile programme, its nuclear activities beyond 2025 - when pivotal provisions of the deal expire - and its role in the wars in Syria and Yemen.\nBritish Foreign Secretary Boris Johnson, in Washington for talks this week, said the deal had weaknesses but these could be remedied.\n“At this moment Britain is working alongside the Trump administration and our French and German allies to ensure that they are,” he said in a commentary in the New York Times. Reporting by Doina Chiacu; Editing by Alistair Bell')

UUID('8e9c51cd-4fef-54bd-b092-74512b579909')

In [4]:
_hash_string_to_uuid('"All money you receive for volunteering or win on a trivia app or lottery is considered income by the IRS," said Mark Kantrowitz, a student loan expert.\nHere are some of the ways to get other people to pay off your debt.\n1) At your job Currently, just 4 percent of employers offer student debt assistance. But that\'s changing as more employers come to realize education debt is a problem for many of their workers, said Katie Berliner, account executive at YouDecide, a benefits firm.')

UUID('8f2d4ce1-0126-5a40-969a-b83d9e9f410e')

In [5]:
from collections import defaultdict

class Document:
    def __init__(self, content_hash, page_content):
        self.content_hash = content_hash
        self.page_content = page_content

def filter_documents(documents):
    hash_map = defaultdict(list)
    for doc in documents:
        hash_map[doc.content_hash].append(doc)
    
    result = []
    for content_hash, docs in hash_map.items():
        if len(docs) > 1:
            print(f"Found multiple documents with the same content hash: {docs[0].page_content[:30]}...")
        result.append(docs[0])
    
    return result

# Tests
def test_filter_documents():
    docs = [
        Document('hash1', 'This is the first document.'),
        Document('hash2', 'This is the second document.'),
        Document('hash1', 'This is a duplicate of the first document.'),
        Document('hash3', 'This is the third document.'),
        Document('hash2', 'This is another duplicate of the second document.'),
    ]
    
    filtered_docs = filter_documents(docs)
    
    assert len(filtered_docs) == 3, f"Expected 3 documents, got {len(filtered_docs)}"
    assert filtered_docs[0].content_hash == 'hash1'
    assert filtered_docs[1].content_hash == 'hash2'
    assert filtered_docs[2].content_hash == 'hash3'
    print("All tests passed.")

# Run tests
test_filter_documents()


Found multiple documents with the same content hash: This is the first document....
Found multiple documents with the same content hash: This is the second document....
All tests passed.
